# Train a Bird Classification Model

Follow steps below to get started with a jupyter notebook for how to train a Towhee operator. This example fine-tunes a pretrained ResNet model (eg. resnet-34 pretrained by ImageNet) with a bird dataset.

## 1. Download Operator

Download operator files together with the jupyter notebook.

In [ ]:
! git clone https://towhee.io/towhee/resnet-image-embedding.git
! cd resnet-image-embedding
! ls

Then run Python scripts in following steps to train and test a Towhee operator.

## 2. Setup Operator

Create operator and load model by name.

In [ ]:
# import sys
# sys.path.append('..')
from resnet_image_embedding import ResnetImageEmbedding

# Set num_classes=400 for ResNet34 model (400 classes of birds in total)
op = ResnetImageEmbedding('resnet34', num_classes=400)

## 3. Configure Trainer:

Modify training configurations on top of default values.

In [ ]:
from towhee.trainer.training_config import TrainingConfig

training_config = TrainingConfig()
training_config.batch_size = 32
training_config.epoch_num = 2
training_config.output_dir = 'bird_output'

## 4. Prepare Dataset

Download [BIRDS 400](https://www.kaggle.com/gpiosenka/100-bird-species) from Kaggle Dataset. And then create the dataset with local path and transform.

In [ ]:
import os
from torchvision import transforms
from torchvision.datasets import ImageFolder

bird_400_path = '/home/zhangchen/zhangchen_workspace/dataset/bird_400'
# bird_400_path = '/path/to/your/dataset/bird_400/'

std = (0.229, 0.224, 0.229)
mean = (0.485, 0.456, 0.406)
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=mean, std=std),
                                transforms.RandomHorizontalFlip(p=0.5)
                               ])
train_data = ImageFolder(os.path.join(bird_400_path, 'train'), transform=transform)
eval_data = ImageFolder(os.path.join(bird_400_path, 'valid'), transform=transform)

## 5. Start Training

Now start training the operator with Bird-400 dataset.

In [ ]:
op.train(training_config, train_dataset=train_data, eval_dataset=eval_data)

## 6. Predict

With the fine-tuned model, you can then use the operator to classify a bird picture.

In [ ]:
from towhee.trainer.utils.plot_utils import predict_image_classification
import random
import matplotlib.pyplot as plt

img_index = random.randint(0, len(eval_data))
img = eval_data[img_index][0]
img_np = img.numpy().transpose(1, 2, 0)  # (C, H, W) -> (H, W, C)
img_np = img_np * std + mean
plt.axis('off')
plt.imshow(img_np)
plt.show()
img_tensor = eval_data[img_index][0].unsqueeze(0).to(op.trainer.configs.device)

prediction_score, pred_label_idx = predict_image_classification(op.model, img_tensor)
print('It is {}.'.format(eval_data.classes[pred_label_idx].lower()))
print('probability = {}'.format(prediction_score))

## 7.Interpret model

If you try to understand why this image will be classified as a mandrin duck, you can use `interpret_image_classification` utils in towhee, which using [captum](https://captum.ai/) as backend. So you must install it first using `pip install captum` or `conda install captum -c pytorch`.

In [ ]:
from PIL import Image
import numpy as np
from towhee.trainer.utils.plot_utils import interpret_image_classification

pil_img = Image.fromarray(np.uint8(img_np * 255))
val_transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean=mean, std=std),
                                    ])
interpret_image_classification(op.model.to('cpu'), pil_img, val_transform, "Occlusion")
interpret_image_classification(op.model.to('cpu'), pil_img, val_transform, "GradientShap")
interpret_image_classification(op.model.to('cpu'), pil_img, val_transform, "Saliency")